<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Vectorización


In [1]:
import numpy as np

In [2]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [3]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [4]:
# Tokenizar cada documento
tokenized_corpus = [doc.split() for doc in corpus]

# Armar un vector de términos no repetidos de todos los documentos
vocabulary = set()
for doc in tokenized_corpus:
    vocabulary.update(doc)

vocabulary = list(vocabulary)  # Convierte el conjunto en una lista

print("Vocabulario del corpus:")
print(vocabulary)

Vocabulario del corpus:
['es', 'hoy', 'gracias', 'dia', 'martes', 'muchas', 'que', 'de', 'el']


### 2- OneHot encoding
Dada una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [5]:
def create_one_hot_matrix(texts):
    # Crear un vocabulario único
    vocabulary = set()
    for text in texts:
        words = text.split()
        vocabulary.update(words)

    # Asignar un índice único a cada palabra en el vocabulario
    word_to_index = {word: i for i, word in enumerate(vocabulary)}

    # Crear la matriz One-Hot Encoding
    num_texts = len(texts)
    num_words = len(vocabulary)
    one_hot_matrix = np.zeros((num_texts, num_words), dtype=int)

    for i, text in enumerate(texts):
        words = text.split()
        for word in words:
            word_index = word_to_index[word]
            one_hot_matrix[i, word_index] = 1

    return one_hot_matrix, vocabulary

In [10]:
# Ejemplo
one_hot_matrix = create_one_hot_matrix(corpus)

print("Matriz One-Hot Encoding:")
print(one_hot_matrix)

Matriz One-Hot Encoding:
(array([[1, 1, 0, 1, 0, 0, 1, 0, 0],
       [1, 1, 0, 1, 1, 0, 0, 1, 1],
       [0, 0, 1, 0, 1, 1, 0, 0, 0]]), {'es', 'hoy', 'gracias', 'dia', 'martes', 'muchas', 'que', 'de', 'el'})


### 3- Vectores de frecuencia
Dada una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [7]:
def create_frequency_matrix(texts):
    # Crear un vocabulario único
    vocabulary = set()
    for text in texts:
        words = text.split()
        vocabulary.update(words)

    # Asignar un índice único a cada palabra en el vocabulario
    word_to_index = {word: i for i, word in enumerate(vocabulary)}

    # Crear la matriz de representación de frecuencia
    num_texts = len(texts)
    num_words = len(vocabulary)
    frequency_matrix = np.zeros((num_texts, num_words), dtype=int)

    for i, text in enumerate(texts):
        words = text.split()
        for word in words:
            frequency_matrix[i, word_to_index[word]] += 1

    return frequency_matrix, vocabulary

In [9]:
# Ejemplo
frequency_matrix = create_frequency_matrix(corpus)

print("Matriz de Representación de Frecuencia:")
print(frequency_matrix)

Matriz de Representación de Frecuencia:
(array([[1, 1, 0, 1, 0, 0, 1, 0, 0],
       [1, 1, 0, 1, 2, 0, 0, 1, 1],
       [0, 0, 1, 0, 1, 1, 0, 0, 0]]), {'es', 'hoy', 'gracias', 'dia', 'martes', 'muchas', 'que', 'de', 'el'})


### 4- TF-IDF
Dada una lista de textos, devolver una matriz con la representacion TFIDF

In [11]:
def create_tfidf_matrix(texts):
    # Crear un vocabulario único y asignar un índice único a cada palabra
    vocabulary = set()
    word_to_index = {}
    for text in texts:
        words = text.split()
        for word in words:
            if word not in vocabulary:
                vocabulary.add(word)
                word_to_index[word] = len(vocabulary) - 1

    # Crear la matriz de términos de documento (DTM)
    num_texts = len(texts)
    num_words = len(vocabulary)
    dtm = np.zeros((num_texts, num_words), dtype=int)

    for i, text in enumerate(texts):
        words = text.split()
        for word in words:
            word_index = word_to_index[word]
            dtm[i, word_index] += 1

    # Calcular la matriz de frecuencia inversa de documentos (IDF)
    df = np.sum(dtm > 0, axis=0)
    idf = np.log((1 + num_texts) / (1 + df)) + 1

    # Calcular la matriz TF-IDF
    tfidf = dtm * idf

    return tfidf, vocabulary

In [12]:
# Ejemplo de uso
tfidf_matrix = create_tfidf_matrix(corpus)

print("Matriz de Representación TF-IDF:")
print(tfidf_matrix)

Matriz de Representación TF-IDF:
(array([[1.69314718, 1.28768207, 1.28768207, 1.28768207, 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 1.28768207, 1.28768207, 1.28768207, 2.57536414,
        1.69314718, 1.69314718, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 1.28768207,
        0.        , 0.        , 1.69314718, 1.69314718]]), {'es', 'hoy', 'gracias', 'dia', 'martes', 'muchas', 'que', 'de', 'el'})


### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [22]:
def compare_cos(document, method = 'one_hot_encode'):
    # Codifica el documento
    if method == 'one_hot_encode':
        matrix,_ = create_one_hot_matrix(document)
    if method == 'frequency_representation':
        matrix,_ = create_frequency_matrix(document)
    if method == 'tfidf_representation':
        matrix,_ = create_tfidf_matrix(document)

    similitud = []
    # Realiza la comparación de cada elemento y lo guarda en 'similitud'
    for i in range(len(document)-1):
        for j in range(i,len(document)):
            if i!=j:
                similarity = cosine_similarity(matrix[i], matrix[j])
                similitud.append([i , j , similarity])
    similitud = np.array(similitud)

    # Imprime los resultados
    for i in similitud:
        print(f'doc:{int(i[0])} con doc:{int(i[1])} tiene similitud de coseno de: {i[2]}')
    
    # Ordena los resultados por similitud de a pares
    similitud1 = similitud[similitud[:, 2].argsort()][::-1][:,:2].flatten()
    indexes = np.unique(similitud1, return_index=True)[1]
    index = [similitud1[index] for index in sorted(indexes)]

    print('\nOrden por similitud de mayor a menor:')
    for i in index:
        print(f'Documento {int(i)}: {document[int(i)]}')

In [23]:
# Ejemplo 
compare_cos(corpus, 'one_hot_encode')

doc:0 con doc:1 tiene similitud de coseno de: 0.6123724356957946
doc:0 con doc:2 tiene similitud de coseno de: 0.0
doc:1 con doc:2 tiene similitud de coseno de: 0.23570226039551587

Orden por similitud de mayor a menor:
Documento 0: que dia es hoy
Documento 1: martes el dia de hoy es martes
Documento 2: martes muchas gracias


In [24]:
compare_cos(corpus, 'frequency_representation')

doc:0 con doc:1 tiene similitud de coseno de: 0.5
doc:0 con doc:2 tiene similitud de coseno de: 0.0
doc:1 con doc:2 tiene similitud de coseno de: 0.3849001794597505

Orden por similitud de mayor a menor:
Documento 0: que dia es hoy
Documento 1: martes el dia de hoy es martes
Documento 2: martes muchas gracias


In [25]:
compare_cos(corpus, 'tfidf_representation')

doc:0 con doc:1 tiene similitud de coseno de: 0.4265993998718088
doc:0 con doc:2 tiene similitud de coseno de: 0.0
doc:1 con doc:2 tiene similitud de coseno de: 0.2929195072289775

Orden por similitud de mayor a menor:
Documento 0: que dia es hoy
Documento 1: martes el dia de hoy es martes
Documento 2: martes muchas gracias
